In [ ]:
import glob  #glob 모듈의 glob 함수는 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환한다
import tensorflow

print(tensorflow.__version__)

# Step 1. 데이터 다운로드

# Step 2. 데이터 읽어오기

# Step 3. 데이터 정제

# Step 4. 평가 데이터셋 분리

# Step 5. 인공지능 만들기